# **INTELIGENCIA ARTIFICIAL APLICADA A LA CIBERSEGURIDAD**
## **PRÁCTICA P1 - BLOQUE I**

**INSTRUCCIONES / RECOMENDACIONES**

- Se recomienda leer con detalle la descripción de cada una de las celdas.
- Las celdas que ya tienen código, se deberán ejecutar directamente.
- Las celdas que están vacías, se completarán con la implementación requerida en el notebook.
- No se incluirán más celdas de las establecidas en el presente notebook, por lo que la solución al mismo deberá implementarse exclusivamente en las celdas vacías.
- Scikit-Learn es un paquete muy útil para las operaciones de preprocesamiento de los datos, como estandarización, normalización, codificación y performance de los modelos.
- Si ves que un apartado es complejo, intenta escribir y ejecutarlo de forma simplificada (por ejemplo, con menos layers o con menos features) y después vaya ampliándolo.
- La entrega se realizará vía Moodle. Será necesario subir la solución a este notebook con el nombre: **NOMBRE_GRUPO.ipynb**

- **Fecha de Publicación: 12/02/2024**
- **Fecha de Entrega: 18/02/2024**
- **Test: 19/02/2024**


# Carga de librerías

In [1]:
import os
import torch
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

print(torch.__version__)

2.1.0+cu121


# Carga dataset

Se va a usar un dataset ('**creditcard.csv**') que contiene información sobre transacciones económicas, en donde cada una de las transacciones está etiquetada como caso de fraude o caso de no fraude.
El objetivo será construir un modelo MLP que permita detectar transacciones fraudulentas, tratándose por lo tanto de un problema de clasificación binaria.

El dataset se puede descargar desde el siguiente enlace:
https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud?resource=download


In [2]:
# Download data from repository and unzip
!wget https://github.com/Laboratorios-2-MIT-MC/IA-Ciber/raw/main/Pr%C3%A1cticas/Pr%C3%A1ctica%201/creditcard.zip -O creditcard.zip
!unzip creditcard.zip

--2024-02-13 18:17:29--  https://github.com/Laboratorios-2-MIT-MC/IA-Ciber/raw/main/Pr%C3%A1cticas/Pr%C3%A1ctica%201/creditcard.zip
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Laboratorios-2-MIT-MC/IA-Ciber/main/Pr%C3%A1cticas/Pr%C3%A1ctica%201/creditcard.zip [following]
--2024-02-13 18:17:30--  https://raw.githubusercontent.com/Laboratorios-2-MIT-MC/IA-Ciber/main/Pr%C3%A1cticas/Pr%C3%A1ctica%201/creditcard.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 69155672 (66M) [application/zip]
Saving to: ‘creditcard.zip’

creditcard.zip      100%[===================>]  65.95M   359MB/s    in 0.

# Preparación de los dataset de Train, Validation y Test

Se deberán crear los dataset **train_loader**, **val_loader** y **test_loader** (Entrenamiento, validación y test), especificando el parámetro **batch_size** que se empleará posteriormente durante el entrenamiento.

In [3]:
# DEFINE & CREATE DATASET

class CreditcardDataset(Dataset):

    def __init__(self, csv_file, transform=None):

        self.df = pd.read_csv(csv_file, sep=",")

        y = self.df.iloc[:]['Class'].values
        x = self.df.drop('Class', axis=1).values

        self.X = torch.tensor(x, dtype=torch.float32)
        self.Y = torch.tensor(y)

        self.transform = transform

        print(self.df.iloc[:]['Class'].value_counts())

    def __len__(self):
        return len(self.Y)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

CC_dataset = CreditcardDataset(csv_file='/content/creditcard.csv')



# PREPARE DATASETS
# Dataset balanceado

from torch.utils.data import TensorDataset, DataLoader, WeightedRandomSampler
from tqdm import tqdm

batch_size = 512
train_split = 0.8 # 80%
val_split = 0.5 # 50% del resto

train_size = int(train_split * len(CC_dataset))
val_size = int(val_split * (len(CC_dataset) - train_size))
test_size = len(CC_dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(CC_dataset, [train_size, val_size, test_size])

# WeightedRandomSampler for TRAIN
class_0 = 0
class_1 = 0
train_labels = []
for i in range(len(train_dataset)):
    _, Y = train_dataset[i]

    if Y==0:
        class_0+=1
    else:
        class_1+=1

    train_labels.append(Y)

print ('TRAIN NO_FRAUD: ', class_0)
print ('TRAIN FRAUD:    ', class_1)

class_counts = [class_0, class_1]
num_samples = sum(class_counts)
labels = train_labels #train_dataset.Y #corresponding labels of samples
class_weights = [num_samples/class_counts[i] for i in range(len(class_counts))]
weights = [class_weights[labels[i]] for i in range(int(num_samples))]
train_sampler = WeightedRandomSampler(torch.DoubleTensor(weights), int(num_samples))

# WeightedRandomSampler for VAL
class_0 = 0
class_1 = 0
val_labels = []
for i in range(len(val_dataset)):
    _, Y = val_dataset[i]

    if Y==0:
        class_0+=1
    else:
        class_1+=1

    val_labels.append(Y)

print ('VAL NO_FRAUD: ', class_0)
print ('VAL FRAUD:    ', class_1)

class_counts = [class_0, class_1]
num_samples = sum(class_counts)
labels = val_labels
class_weights = [num_samples/class_counts[i] for i in range(len(class_counts))]
weights = [class_weights[labels[i]] for i in range(int(num_samples))]
val_sampler = WeightedRandomSampler(torch.DoubleTensor(weights), int(num_samples))

# WeightedRandomSampler for TEST
class_0 = 0
class_1 = 0
test_labels = []
for i in range(len(test_dataset)):
    _, Y = test_dataset[i]

    if Y==0:
        class_0+=1
    else:
        class_1+=1

    test_labels.append(Y)

print ('TEST NO_FRAUD: ', class_0)
print ('TEST FRAUD:    ', class_1)

class_counts = [class_0, class_1]
num_samples = sum(class_counts)
labels = test_labels #train_dataset.Y #corresponding labels of samples
class_weights = [num_samples/class_counts[i] for i in range(len(class_counts))]
weights = [class_weights[labels[i]] for i in range(int(num_samples))]
test_sampler = WeightedRandomSampler(torch.DoubleTensor(weights), int(num_samples))


train_loader = DataLoader(train_dataset, batch_size=batch_size,
                            #shuffle=True,
                            num_workers=0,
                            sampler=train_sampler
                            )
val_loader = DataLoader(val_dataset, batch_size=batch_size,
                            #shuffle=True,
                            num_workers=0,
                            sampler=val_sampler
                            )
test_loader = DataLoader(test_dataset, batch_size=batch_size,
                            #shuffle=True,
                            num_workers=0,
                            sampler=test_sampler
                            )

print ("# total samples:", len(CC_dataset))
print ("# total TRAIN samples:", len(train_loader.sampler))
print ("# total VAL samples:", len(val_loader.sampler))
print ("# total TEST samples:", len(test_loader.sampler))

0    284315
1       492
Name: Class, dtype: int64
TRAIN NO_FRAUD:  227459
TRAIN FRAUD:     386
VAL NO_FRAUD:  28419
VAL FRAUD:     62
TEST NO_FRAUD:  28437
TEST FRAUD:     44
# total samples: 284807
# total TRAIN samples: 227845
# total VAL samples: 28481
# total TEST samples: 28481


# Análisis del dataset

Realice un análisis de las variables en el dataset.
Sugerencias:
- Histogramas de las variables.
- Obtención de valores de centralidad y dispersión.
- Detectar variables que no parezcan predecir el target o si presentan alta correlación con alguna otra de las features.



# Activación de GPU

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
torch.cuda.get_device_name(0)

# Creación del modelo MLP

Se creará un modelo MLP denominado **model** que implementará un clasificador binario para la detección de transacciones fraudulentas.

# Definición de función de pérdida y optimizador

Especificar el parámetro **lr** (learning rate) que se empleará durante el entrenamiento del modelo.

# Definición de funciones de Entrenamiento y Validación

In [ ]:
def train_model(model, train_loader, criterion, optimizer, epoch):

    model.train()

    total_epoch_loss = 0
    total_epoch_acc = 0

    for i, (X, Y) in enumerate(train_loader):

        X, Y = X.to(device), Y.to(device)

        outputs = model(X)
        loss = criterion(outputs, Y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        num_corrects = (torch.max(outputs, 1)[1].view(Y.size()).data == Y.data).float().sum()
        acc = 100.0 * num_corrects/len(Y)

        if (i+1) % 1000 == 0:
          print (f'Epoch: {epoch+1}, Idx: ({i+1}/{len(train_loader)}), Training Loss: {loss.item():.4f}, Training Accuracy: {acc.item(): .2f}%')

        total_epoch_loss += loss.item()
        total_epoch_acc += acc.item()

    return total_epoch_loss/len(train_loader), total_epoch_acc/len(train_loader)

def eval_model(model, val_loader, criterion):

  model.eval()

  total_epoch_loss = 0
  total_epoch_acc = 0

  with torch.no_grad():

    for i, (X, Y) in enumerate(val_loader):

        X, Y = X.to(device), Y.to(device)

        outputs = model(X)
        loss = criterion(outputs, Y)

        num_corrects = (torch.max(outputs, 1)[1].view(Y.size()).data == Y.data).float().sum()
        acc = 100.0 * num_corrects/len(Y)

        total_epoch_loss += loss.item()
        total_epoch_acc += acc.item()

    return total_epoch_loss/len(val_loader), total_epoch_acc/len(val_loader)

# Entrenamiento del modelo

Especificar el parámetro **epochs** que se utilizará durante el entrenamiento del modelo.

In [ ]:
train_loss_epochs = []
val_loss_epochs = []
train_acc_epochs = []
val_acc_epochs = []

for epoch in range(epochs):
    train_loss, train_acc = train_model(model, train_loader, criterion, optimizer, epoch)
    val_loss, val_acc = eval_model(model, val_loader, criterion)

    print(f'Epoch: [{epoch+1:02}/{epochs}], Train Loss: {train_loss:.3f}, Train Acc: {train_acc:.2f}%, Val. Loss: {val_loss:3f}, Val. Acc: {val_acc:.2f}%')
    train_loss_epochs.append(train_loss)
    val_loss_epochs.append(val_loss)
    train_acc_epochs.append(train_acc)
    val_acc_epochs.append(val_acc)

# Mostrar la evolución del error y del accuracy durante el entrenamiento del modelo

Para mostrar el error y accuracy del dataset de train y de val, haga uso de las listas creadas durante el entrenamiento del modelo (ver celda anterior).

# Verificar el rendimiento del modelo creado

Utilice el dataset de test para verificar el correcto rendimiento del modelo. Para ello puede emplear las métricas que considere oportunas, como por ejemplo Accuracy, Precision, Recall, F1-score o matriz de confusión.